<a href="https://colab.research.google.com/github/mikedcurry/DS-Unit-2-Regression-Classification/blob/master/Curry_212_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ X] Engineer at least two new features. (See below for explanation & ideas.)
- [X ] Fit a linear regression model with at least two features.
- [X ] Get the model's coefficients and intercept.
- [ X] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [X] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [X ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [59]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.0.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [9]:
df.head(2)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df.shape

(48817, 34)

In [0]:
#All the libraries I might need...

#Visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D

#Regession model
from sklearn.linear_model import LinearRegression

#Maybe for fun
from ipywidgets import interact

# Split Data T/T
from sklearn.model_selection import train_test_split

# Inspect Error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [0]:
# First I want to engineer a new feature: my plan is to sum the aminities into a single column



In [30]:
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                   int64
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [0]:
# Combine all amenities to a total column

df['amenities'] = df['elevator']+ df['cats_allowed']+ df['hardwood_floors']
+ df['dogs_allowed']+ df['doorman']+ df['dishwasher']+ df['no_fee']+ df['laundry_in_building']
+ df['fitness_center']+ df['pre-war']+ df['laundry_in_unit']+ df['roof_deck']+ df['outdoor_space']
+ df['dining_room']+ df['high_speed_internet']+ df['balcony']+ df['swimming_pool']
+ df['new_construction']+ df['terrace']+ df['exclusive']+ df['loft']+ df['garden_patio']
+ df['wheelchair_access']+ df['common_outdoor_space']

In [64]:
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,amenities
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Change created to a time value so I can split the data

df['created'] = pd.to_datetime(df['created'])

In [26]:
# Inspect values inside interest level. Going to use a dictionary to replace with numberical data 

df['interest_level'].unique()

array(['medium', 'low', 'high'], dtype=object)

In [28]:
# inspect distribution

df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [0]:
df.interest_level = df.interest_level.replace({'low':1, 'medium':2, 'high':3})

In [31]:
df.head(2)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,amenities
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [0]:
df.isnull().sum()

In [0]:
# Okay Now I'm ready to split, fit and test

# My linear regression features will be: bedrooms, bathrooms, interest level, amenities

# Wont be able to graph it, but I can get my values at the end...

In [0]:
df_train = df[(df['created'] < '2016-06')]

In [72]:
df_train.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,amenities
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
df_test = df[~(df['created'] < '2016-06')]

In [57]:
df_test['created'].describe()

count                   16973
unique                  16712
top       2016-06-25 01:30:16
freq                        3
first     2016-06-01 01:10:37
last      2016-06-29 21:41:47
Name: created, dtype: object

In [55]:
df_train['created'].describe()

count                   31844
unique                  31436
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-05-31 23:10:48
Name: created, dtype: object

In [0]:
X_train2 = df_train[['bedrooms', 'bathrooms', 'interest_level', 'amenities']]
y_train2 = df_train['price']

X_test2 = df_test[['bedrooms', 'bathrooms', 'interest_level', 'amenities']]
y_test2 = df_test['price']

In [0]:
#Name my model 

regressify = LinearRegression()

In [77]:
# Fit dat shhh--tuff

regressify.fit(X_train2, y_train2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [78]:
print('Train R^2 score:', regressor.score(X_train2, y_train2))
print('Test R^2 score:', regressor.score(X_test2, y_test2))

Train R^2 score: 0.5559529666900401
Test R^2 score: 0.5677265500553974


In [80]:
# Alright! Now I've got the coefficients of my 5th demensional regression. woo!

regressify.intercept_, regressify.coef_

(946.9140337819722,
 array([ 430.125661  , 1909.29645326, -502.70972306,  247.47084257]))

In [0]:
#  Get the model's coefficients and intercept.
#  Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.

In [0]:
# Setting up my equation for my 4D plane

#['bedrooms', 'bathrooms', 'interest_level', 'amenities']
y = df['price']

x1 = df['bedrooms']
x2 = df['bathrooms']
x3 = df['interest_level']
x4 = df['amenities']

y_preds = 946.9 + 430.1*x1 + 1909.3*x2 - 502.7*x3 + 247.5*x4 

In [85]:
# Print regression metrics

mse = mean_squared_error(y, y_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, y_preds)
r2 = r2_score(y, y_preds)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1366602.856213409
Root Mean Squared Error: 1169.0179024349495
Mean Absolute Error: 770.751391933138
R^2: 0.5600259253923512


In [0]:
# For comparison, I'm running a random split below...


X = df[['bedrooms', 'bathrooms', 'interest_level', 'amenities']] # Matrix features
y = df['price']                                                  # Vector Target

In [0]:
# Split Data into T/Ts (what? it's my numonic)

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Can't tell what my output is? Is it an array? 

In [0]:
# Name my model
regressor = LinearRegression()

In [37]:
# Fit my Data.

regressor.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [39]:
print('Train R^2 score:', regressor.score(X_train, y_train))
print('Test R^2 score:', regressor.score(X_test, y_test))

Train R^2 score: 0.5628774669149337
Test R^2 score: 0.5516353316536706
